In [1]:
import os
import numpy as np
#np.random.seed(37)
import random 
#random.seed(1254)
import  pickle
import re

In [2]:
model_version=1520689

In [3]:
import tensorflow as tf

#import tensorflow.compat.v1 as tf
#tf.set_random_seed(89)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
             tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 

1 Physical GPUs, 1 Logical GPUs


In [4]:
model = tf.keras.models.load_model('Models/dblp_db_model_new'+str(model_version)+'.h5')

In [5]:
import chars2vec
#with tf.device('/gpu:0'):
import random
chars2vec_model = chars2vec.load_model('eng_200')
from sentence_transformers import SentenceTransformer
bert_model = SentenceTransformer('distilbert-base-nli-mean-tokens')
import sqlite3


import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import json
import string
import itertools
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from datetime import datetime
from matplotlib import pyplot as plt
#with tf.device('/gpu:0'):
db_name = 'Source/dblp.db'
global conn
conn = sqlite3.connect(db_name,check_same_thread=False) 

In [6]:
# Load train, test and validation daataset
#with tf.device('/gpu:0'):
with open('Training/Miscs/new_authors_ids_'+str(model_version)+'.pickle','rb') as file:
    new_authors_ids=pickle.load(file)
with open('Training/Miscs/old_authors_ids_'+str(model_version)+'.pickle','rb') as file:
    old_authors_ids=pickle.load(file)
with open('Training/Data/x_test_new_reshuffeled_'+str(model_version)+'.pickle','rb') as file:
    x_test=pickle.load(file)
with open('Training/Data/y_test_new_reshuffeled_'+str(model_version)+'.pickle','rb') as file:
    y_test=pickle.load(file)
with open('Source/auth2index.pickle','rb') as file:
    auth2index=pickle.load(file)

In [7]:
old_authors_ids=list(old_authors_ids)
new_authors_ids=list(new_authors_ids)

In [8]:



def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

try:
    with open('Training/Miscs/records_test_'+str(model_version)+'.pickle', 'rb') as handle:
        records_test=pickle.load(handle)
    with open('Training/Miscs/ref_test_'+str(model_version)+'.pickle', 'rb') as handle:
        ref_test=pickle.load(handle)
except:
    

    conn.row_factory = dict_factory
    query = "SELECT * FROM reference WHERE combination_id in "+str(tuple(x_test))
    cursor = conn.cursor()
    cursor.execute(query)
    records_test = cursor.fetchall()
    cursor.close()
    with open('Training/Miscs/records_test_'+str(model_version)+'.pickle', 'wb') as handle:
        pickle.dump(records_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
       
    
    
    ref_test=[]
    for record in records_test:
        ref_test.append(tuple(record.values())[1])
    ref_test=np.array(ref_test)
 
    with open('Training/Miscs/ref_test_'+str(model_version)+'.pickle', 'wb') as handle:
        pickle.dump(ref_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
idx=5
print(records_test[idx])
print(x_test[idx])
print(y_test[idx])
tmp=auth2index[records_test[idx]['target_author']]
target_author_ind=new_authors_ids[old_authors_ids.index(tmp)]
print (target_author_ind)


{'combination_id': 98371, 'reference_id': 9870, 'target_author': 'Yujie Wang', 'author_fname': 'Yujie', 'author_lname': 'Wang', 'coauthor_fname': 'Mikhail', 'coauthor_lname': 'M. Svinin', 'title': 'Function Approximation Technique Based Immersion and Invariance Control for Unknown Nonlinear Systems.', 'year': '2020', 'abbr_journal': 'IEEE Control. Syst. Lett.', 'journal': ''}
98371
2287
2287


In [10]:
def generate_name_variants(auth_fname,auth_lname,coauth_fname,coauth_lname,coauth2_fname,coauth2_lname):

    # Remove all the punctuations and convert to lowercase
    auth_fname = auth_fname.translate(str.maketrans('', '', string.punctuation)).casefold()
    auth_lname = auth_lname.translate(str.maketrans('', '', string.punctuation)).casefold()
    coauth_fname = coauth_fname.translate(str.maketrans('', '', string.punctuation)).casefold()
    coauth_lname = coauth_lname.translate(str.maketrans('', '', string.punctuation)).casefold()
    coauth2_fname = coauth2_fname.translate(str.maketrans('', '', string.punctuation)).casefold()
    coauth2_lname = coauth2_lname.translate(str.maketrans('', '', string.punctuation)).casefold()

    auth_name = auth_fname
    coauth_name = coauth_fname + " " + coauth_lname
    coauth2_name = coauth2_fname + " " + coauth2_lname

    combo_list = []
    auth_name_variants = []
    coauth_name_variants = []
    coauth2_name_variants = []
    


    auth_name_variants.append(auth_fname)
    auth_name_variants.append(' '.join([tmp[0] for tmp in auth_fname.split()]))


    coauth_name_variants.append(coauth_name)
    coauth_name_variants.append(' '.join([tmp[0] for tmp in coauth_fname.split()])+ " " + coauth_lname)


    coauth2_name_variants.append(coauth2_name)
    coauth2_name_variants.append(' '.join([tmp[0] for tmp in coauth2_fname.split()])+ " " + coauth2_lname)


    combo_list = list(zip(auth_name_variants,coauth_name_variants,coauth2_name_variants))
    return combo_list  

In [11]:
def get_emb(combo):
    
    author_name = combo[0]
    
    try:
        with open('Training/tdata/auth_data/'+author_name+'.pickle','rb') as file:
            auth_emb=pickle.load(file)
    except:
        auth_emb = chars2vec_model.vectorize_words([author_name])[0]
        with open('Training/tdata/auth_data/'+author_name+'.pickle','wb') as file:
            pickle.dump(auth_emb,file)
            
            
    co_author_name1 = combo[1]
    
    try:
        with open('Training/tdata/auth_data/'+co_author_name1+'.pickle','rb') as file:
            co_auth_emb1=pickle.load(file)
    except:
        co_auth_emb1 = chars2vec_model.vectorize_words([co_author_name1])[0]
        with open('Training/tdata/auth_data/'+co_author_name1+'.pickle','wb') as file:
            pickle.dump(co_auth_emb1,file)
            
    co_author_name2 = combo[2]
    
    try:
        with open('Training/tdata/auth_data/'+co_author_name2+'.pickle','rb') as file:
            co_auth_emb2=pickle.load(file)
    except:
        co_auth_emb2 = chars2vec_model.vectorize_words([co_author_name2])[0]
        with open('Training/tdata/auth_data/'+co_author_name2+'.pickle','wb') as file:
            pickle.dump(co_auth_emb2,file)

    return np.concatenate([np.sum((auth_emb,auth_emb,auth_emb), axis=0),np.sum((co_auth_emb1,co_auth_emb2), axis=0)])
    #return np.concatenate([co_auth_emb1,co_auth_emb2])
    
def get_emb2(record):
    title = record[7]
    # take journal full form else abbr journal if full form not avaialable
    journal = record[10] if len(record[10])>0 else record[9]
    
    content= journal+' '+title
    
    
    try:
        with open('Training/tdata/ref_data/title'+str(record[1])+'.pickle','rb') as file:
            content_emb=pickle.load(file)
    except:
        content_emb = bert_model.encode(content)
        with open('Training/tdata/ref_data/title'+str(record[1])+'.pickle','wb') as file:
            pickle.dump(content_emb,file)
            
        
    return content_emb

def generate_embeddings2(record,record2,record3):   
    embeddings_list = []
    embeddings_list2 = []
    labels_list = []
    
    #there were a mistake in the obtention of fname and lname
    a0=record[3]+' '+record[4]
    a0=re.sub('[0-9\-\"]',' ',a0)
    a0=a0.split()
    a0_fname=' '.join(a0[:-1])
    a0_lname=a0[-1]
    if not bool(a0_fname):
        a0_fname=a0_lname
        
    a1=record[5]+' '+record[6]
    a1=re.sub('[0-9\-\"]',' ',a1)
    a1=a1.split()
    a1_fname=' '.join(a1[:-1])
    a1_lname=a1[-1]
    if not bool(a1_fname):
        a1_fname=a1_lname
    
    a2=record2[5]+' '+record2[6]
    a2=re.sub('[0-9\-\"]',' ',a2)
    a2=a2.split()
    a2_fname=' '.join(a2[:-1])
    a2_lname=a2[-1]
    if not bool(a2_fname):
        a2_fname=a2_lname
    
    # Get the combinations of name variants
    combo_list = generate_name_variants(a0_fname,a0_lname,a1_fname,a1_lname,a2_fname,a2_lname)

    # Get the target author index
    target_author_name = record[2]
    tmp=auth2index[target_author_name]
    target_author_ind=new_authors_ids[old_authors_ids.index(tmp)]
    #target_author_ind = auth2index[target_author_name]
    
    tmp=[(embeddings_list.append(get_emb(combo)), embeddings_list2.append(get_emb2(record3))) for combo in combo_list]
    labels_list=[target_author_ind]*len(embeddings_list)

    embeddings_list = np.array(embeddings_list)
    embeddings_list2 = np.array(embeddings_list2)
    labels_list = np.array(labels_list)
    embeddings_list = tf.keras.utils.normalize(embeddings_list, axis = 1)
    embeddings_list2 = tf.keras.utils.normalize(embeddings_list2, axis = 1)

    return embeddings_list,embeddings_list2,labels_list

In [ ]:
r=[]
u_ref=np.unique(ref_test)
for u in u_ref:
    print (u)
    tmp=np.where(ref_test==u)[0]
    x=itertools.combinations(tmp, 2)
    a1=[]
    a2=[]
    
    for xx in tmp:
        feature_id1=tmp[0]
        record=tuple(records_test[feature_id1].values())
        record2=record
        record3=record
        e1,e2,l=generate_embeddings2(record,record2,record)

        v1=[np.array([e1[0]]),np.array([e2[0]])]
        v2=[np.array([e1[1]]),np.array([e2[1]])]
        a1=model.predict(v1)[0]+a1 if len(a1)>0 else model.predict(v1)[0]
        a2=model.predict(v2)[0]+a2 if len(a2)>0 else model.predict(v2)[0]
    
    
    for xx in x:
        feature_id1=xx[0]
        feature_id2=xx[1]
        record=tuple(records_test[feature_id1].values())
        record2=tuple(records_test[feature_id2].values())
        record3=record
        e1,e2,l=generate_embeddings2(record,record2,record)

        v1=[np.array([e1[0]]),np.array([e2[0]])]
        v2=[np.array([e1[1]]),np.array([e2[1]])]
        a1=model.predict(v1)[0]+a1 if len(a1)>0 else model.predict(v1)[0]
        a2=model.predict(v2)[0]+a2 if len(a2)>0 else model.predict(v2)[0]
    
    r.append([np.argmax(a1),y_test[feature_id1],u,1])
    r.append([np.argmax(a2),y_test[feature_id1],u,2])

5069
5842
9870
10584
11451
11805
12097
13133
14335
14536
21455
21737
23358
23533
25050
26865
26873
27162
27366
27748
29725
29781
30024
30273
31611
31735
31963
32111
32138
32256
32521
32684
32831
32915
33639
34157
34559
35580
35597
35962
36043
36068
36119
36482
36487
37237
37284
37411
38891
38928
39107
39119
46286
49896
50500
50901
51828
51849
53216
53489
53754
53792
54216
54270
54343
54494
54501
54891
55194
55814
56070
61111
63443
63649
64134
66733
66871
67596
68398
69709
70724
72293
72651
72933
73428
74027
74039
74754
75866
76071
76524
80848
80899
81154
81807
82070
82254
82578
82593
82738
82760
82993
83131
83322
83886
84108
84307
85544
85898
86024
86287
87285
87826
88659
92983
94984
95037
95631
95660
95835
95924
96497
97057
97357
97527
98305
98637
99105
99569
100391
100635
101013
101218
101341
101382
101501
101728
101793
101904
102362
102429
102475
102561
102707
102939
102945
103156
103877
103993
104445
104631
104974
105036
105270
105282
105732
105954
109777
109976
110118
110680
11130

767117
767540
767719
767888
768470
768579
768750
768983
769139
769226
770116
770162
770510
770764
771408
774784
775108
776082
777375
777700
781097
781488
782843
782871
783705
784316
784332
784341
784754
787742
787977
788904
789992
790334
790759
791111
792625
793194
793316
793523
793777
794225
794466
794537
794803
794925
795902
796145
796508
796647
796849
797726
798264
798934
799619
799677
799765
800127
800242
800474
802191
802213
802275
802276
802576
804771
804878
805350
805497
805796
805895
806253
807475
810394
810775
811319
815226
815866
818148
820430
820691
820936
821385
822258
822820
823834
823951
824210
825373
826042
826145
827332
827587
828036
828068
828077
828372
828799
832100
832614
833185
833293
833486
833537
833640
833705
834201
834387
834818
834859
836264
836359
839037
840851
841063
842110
842605
843446
843959
844011
845141
845187
845699
846072
847886
848216
848826
849267
849521
853667
854583
855076
855160
855977
856223
856764
856928
857249
857296
857725
858992
860046
860113

In [ ]:
with open('Models/results_'+str(model_version)+'.pickle','wb') as file:
    pickle.dump(r,file)

In [1]:
x=np.array(r).T

NameError: name 'np' is not defined

In [19]:
#tmp=np.where(x[3]==1)[0]
#xx=np.array(r)[tmp]
#x=np.array(xx).T

In [20]:
import sklearn
a=sklearn.metrics.precision_recall_fscore_support(x[1], x[0],average='macro')
b=sklearn.metrics.precision_recall_fscore_support(x[1], x[0],average='micro')


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
print (a)
print (b)

(0.3631327698175781, 0.3289453726692913, 0.3284501490142097, None)
(0.6061906221268771, 0.6061906221268771, 0.6061906221268771, None)


In [14]:
print ('yes')

yes


In [ ]:
with open('Models/results_'+str(model_version)+'.pickle','rb') as file:
    r=pickle.load(file)

In [ ]:
x=np.array(r).T
tmp=np.where(x[2]==1)[0]
xx=np.array(r)[tmp]
x=np.array(xx).T

In [ ]:
xx[0]

In [ ]:
a=64
print (np.where(np.array(y_test)==a))

In [ ]:
idx=0
print(records_test[idx])
print(x_test[idx])
print(y_test[idx])
tmp=auth2index[records_test[idx]['target_author']]
target_author_ind=new_authors_ids[old_authors_ids.index(tmp)]
print (target_author_ind)
